***Прогнозирование стоимости автомобиля по характеристикам***

Установка и импорт всех необходимых библиотек

In [1]:
import numpy as np 
import pandas as pd 

import re
import sys
import itertools
import datetime
from tqdm.notebook import tqdm
import pandas_profiling

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer

from catboost import CatBoostRegressor
import xgboost as xgb

In [2]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [4]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [5]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

Загрузка данных и предварительный анализ

In [6]:
VERSION    = 16
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

In [7]:
!ls '../input'

In [8]:
train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

***Посмотрим на данные***

In [9]:
train.head()

In [10]:
train.info()

In [11]:
test.head()

In [12]:
test.info()

In [13]:
cols_to_remove = []

In [14]:
set(test.columns).difference(train.columns)

***Предварительная обработка данных и EDA***

In [15]:
test.vendor.unique()

In [16]:
vendor_dict = {k: v for v, k in test.groupby(
    ['vendor', 'brand']).name.count().index}
print(vendor_dict)
train['vendor'] = train.brand.map(vendor_dict)
train.vendor.unique()

In [17]:
test.model_info[0]

In [18]:
test.drop(labels='model_info', axis=1, inplace=True)

В тестовом наборе данных у нас нет столбца цен, потому что нам нужно его предсказать. А пока создадим его и заполним нулями.

In [19]:
test['price'] = 0.0

Мы можем проверить бренды, которые присутствуют в наборе данных train, и исключить все бренды, которые не включены в набор данных test.

In [20]:
train.brand.unique()

In [21]:
train = train[train.brand.isin(test.brand.unique())]

In [22]:
train.brand.unique()

In [23]:
train.rename(columns={'model': 'model_name', 'Комплектация': 'complectation_dict'}, inplace=True)

Приведем данные к одному формату

In [24]:
train.engineDisplacement.unique()

In [25]:
def convert_engineDisplacement_to_float(row):
    row = str(row)
    volume = re.findall('\d\.\d', row)
    if volume == []:
        return None
    return float(volume[0])

In [26]:
train.engineDisplacement = train.name.apply(convert_engineDisplacement_to_float)
train.engineDisplacement.unique()

In [27]:
train['train'] = 1
test['train'] = 0
train['sell_id'] = 0

Объединяем все фреймы данных

In [28]:
df_com = pd.concat([test, train], join='inner', ignore_index=True)
df_com.head()

In [29]:
df_com.columns

In [30]:
df_com.rename(columns={'Владельцы': 'owners', 'Владение': 'ownership', 'ПТС': 'vehicle_licence',
       'Привод': 'driving_gear', 'Руль': 'steering_wheel', 'Состояние': 'condition', 'Таможня': 'customs'}, inplace=True)

In [31]:
df_com.isna().sum()

In [32]:
pandas_profiling.ProfileReport(df_com)

***Резюме***
- Набор данных состоит из 27 признаков и 83995 объектов
- Процент пропущенных значений - 6.5% (сделаем позже)
- В наборе данных нет дубликатов
- Перед любой предварительной обработкой есть 18 категориальных столбцов, 5 числовы и 4 столбца имеют неподдерживаемый тип данных (проверим позже)
- Существует сильная корреляция между некоторыми столбцами: productionDate и modelDate, vendor и brand.

***Обработка признаков***

In [33]:
df_com.bodyType = df_com.bodyType.apply(lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)

In [34]:
df_com.bodyType.value_counts(normalize=True)

In [35]:
df_com.color.value_counts()

In [36]:
color_dict = {'040001': 'чёрный', 'FAFBFB': 'белый', '97948F': 'серый', 'CACECB': 'серебристый', '0000CC': 'синий', '200204': 'коричневый',
              'EE1D19': 'красный',  '007F00': 'зелёный', 'C49648': 'бежевый', '22A0F8': 'голубой', '660099': 'пурпурный', 'DEA522': 'золотистый', 
              '4A2197': 'фиолетовый', 'FFD600': 'жёлтый', 'FF8649': 'оранжевый', 'FFC0CB': 'розовый'}

In [37]:
df_com.color.replace(to_replace=color_dict, inplace=True)
df_com.color.value_counts(normalize=True)

In [38]:
df_com.engineDisplacement.unique()

In [39]:
df_com.engineDisplacement = df_com.engineDisplacement.apply(
    lambda x: x.replace(" LTR", "0.0 LTR") if x == " LTR" else x)
df_com.engineDisplacement = df_com.engineDisplacement.apply(
    lambda x: float(x.replace("LTR", "")) if isinstance(x, str) else x)

In [40]:
df_com.engineDisplacement.unique()

In [41]:
print(df_com.enginePower.unique())
df_com.enginePower = df_com.enginePower.apply(
    lambda x: float(x.replace("N12", "")) if isinstance(x, str) else x)
df_com.enginePower.unique()

In [42]:
print(df_com.owners.unique())
df_com.owners = df_com.owners.apply(lambda x: float(
    re.sub('\D', '', x)) if isinstance(x, str) else x)
df_com.owners.unique()

In [43]:
print(df_com.customs.unique())
df_com.customs = df_com.customs.apply(
    lambda x: 1 if x == 'Растаможен' or x == True else 0)
print(df_com.customs.unique())

In [44]:
df_com.vehicleTransmission.unique()

In [45]:
df_com.vehicleTransmission = df_com.vehicleTransmission.apply(
    lambda x: 'mechanical' if x == 'механическая' or x == 'MECHANICAL' else 'automatic')

In [46]:
df_com.vehicleTransmission.unique()

In [47]:
print(df_com.vehicle_licence.unique())
df_com.vehicle_licence = df_com.vehicle_licence.apply(
    lambda x: 'original' if x == 'Оригинал' or x == 'ORIGINAL' else 'duplicate')
df_com.vehicle_licence.unique()

In [48]:
print(df_com.steering_wheel.unique())
df_com.steering_wheel = df_com.steering_wheel.str.replace(
    'Левый', 'left').replace('Правый', 'right').str.lower()
df_com.steering_wheel.unique()

In [49]:
cols_to_remove.extend(['ownership', 'description', 'vehicleConfiguration'])

***Удаление дублей***

In [50]:
print(sum(df_com.duplicated()))
df_com.shape

In [51]:
df_com.drop_duplicates(inplace=True)
df_com.shape

***Заполнение пропущенных значений***

In [52]:
df_com.isna().sum(axis=0) * 100 / df_com.shape[0]

Удалить 2 столбца: владение и complectation_dict

Столбец цены: в нем отсутствуют некоторые значения, которые можно только удалить, но нельзя заполнить.

In [53]:
df_com[df_com.train == 1].price.isna().sum(), df_com[df_com.train == 0].price.isna().sum()

In [54]:
df_com.dropna(subset=['price'], inplace=True)

In [55]:
df_com.info()

In [56]:
df_com.condition.value_counts()

In [57]:
df_com.condition = df_com.condition.apply(
    lambda x: 1 if x == 'Не требует ремонта' else 0)

In [58]:
cols_to_remove.append('name')

In [59]:
df_com.loc[(df_com.engineDisplacement.isna()) & (df_com.fuelType ==
                                                 'электро'), 'engineDisplacement'] = 0.0  # Для электромобидей

In [60]:
df_com.loc[(df_com.owners.isna()) & (df_com.mileage == 0.0),
           'owners'] = 0  # Для новых авто без владельца

In [61]:
df_com.isna().sum()

In [62]:
df_com.loc[df_com.driving_gear.isna(), 'driving_gear']
df_com.loc[46769]

In [63]:
df_com=df_com.drop(labels = [46769],axis = 0)

***EDA, обнаружение выбросов***

Давайте посмотрим на количество уникальных значений в каждом столбце и на их типы данных, которые могут помочь нам разделить столбцы на категории.

In [64]:
df_com.nunique(dropna=False)

In [65]:
df_com.info()

Сгруппируем колонки по типу данных

In [66]:
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'model_name',
            'name', 'driving_gear', 'owners', 'numberOfDoors']
num_cols = ['engineDisplacement', 'enginePower',
            'mileage', 'modelDate', 'productionDate']
bin_cols = ['condition', 'customs', 'steering_wheel',
            'vehicleTransmission', 'vendor', 'vehicle_licence']
help_cols = ['train', 'sell_id', 'parsing_date']
target_cols = ['price']

all_cols = cat_cols + num_cols + bin_cols + help_cols + target_cols
len(all_cols)

In [67]:
cols_to_remove

***Анализ числовых столбцов: распределение, корреляция, выбросы***

In [68]:
sns.pairplot(df_com[num_cols]);

Глядя на распределение, мы видим, что столбцы modelDate и productionDate сильно коррелированы. В конце нам нужно удалить один из этих столбцов. Во всех случаях мы не видим нормального распределения для значений, а распределения либо смещены вправо, либо влево.

In [69]:
plt.figure(figsize=(15, 8))
sns.heatmap(df_com[df_com.train == 1][num_cols + ['price']
                                      ].corr(), vmin=-1, vmax=1, annot=True, cmap='vlag')

enginePower и engineDisplacement также демонстрируют высокую корреляцию.

productionDate и mileage также показывают высокую отрицательную корреляцию.

In [70]:
cols_to_remove.extend(['modelDate', 'engineDisplacement'])

***Работа с выбросами***

In [71]:
df_com[(df_com.train == 1) & (df_com.enginePower >
                              df_com.query('train == 0').enginePower.max())]

In [72]:
df_com.drop(
    df_com[(df_com.train == 1) & (df_com.enginePower > df_com.query(
        'train == 0').enginePower.max())].index, inplace=True
)

In [73]:
df_com[df_com.productionDate < 1960]

In [74]:
plt.figure(figsize=(25, 6))
sns.scatterplot(data=df_com[df_com['train'] == 1],
                x='productionDate', y="price")

Можно создать дополнительный признак для раритетных автомобилей и для автомобилей старше 5 лет (падение цены)

In [75]:
num_cols = ['enginePower', 'mileage', 'productionDate']

In [76]:
cols_to_remove.extend(['condition', 'customs', 'complectation_dict'])

***Анализ целевой характеристики («цены»)***

In [77]:
df_com.query('train == 1').price.hist();
plt.title('The target variable distribution', fontdict={'fontsize': 14});
plt.xlabel('price, RUB * 10^7');

In [78]:
np.log2(df_com.query('train == 1').price).hist();
plt.title('The log2 target variable distribution', fontdict={'fontsize': 14});

In [79]:
df_com['price_log2'] = np.log2(df_com.price + 1)

***Feature engineering***

In [80]:
# сколько км автомобиль проехал за год
df_com['mileage_per_year'] = df_com.productionDate / df_com.mileage
df_com['mileage_per_year'].replace([np.inf, -np.inf], 0, inplace=True)

In [81]:
# возраст авто
df_com['age_year'] = 2021 - df_com.productionDate
df_com['age_year'].replace([np.inf, -np.inf], 0, inplace=True)

In [82]:
# старше ли авто 5 лет
df_com['older_5y'] = df_com.productionDate.apply(lambda x: 1 if x < 2021 - 5 else 0)

In [83]:
# Кроссовер или седан
df_com['top2_bodyType'] = df_com.bodyType.apply(lambda x: 1 if x in ['внедорожник', 'седан'] else 0)

In [84]:
# выпускался ли автомобиль до 1970 г
df_com['rarity'] = df_com.productionDate.apply(lambda x: 1 if x < 1970 else 0)

***Окончательные столбцы***

In [85]:
df_com.drop(cols_to_remove, axis=1, inplace=True)

In [86]:
df_com.info()

In [87]:
num_cols, bin_cols, cat_cols

In [88]:
bin_cols.extend(['top2_bodyType', 'older_5y', 'rarity'])

***Кодирование категориальных признаков***

In [89]:
for colum in ['steering_wheel', 'vehicleTransmission', 'vendor', 'vehicle_licence']:
    df_com[colum] = df_com[colum].astype('category').cat.codes

cols_to_encode = list(set(df_com.columns) & set(cat_cols))
for colum in cols_to_encode:
    df_com[colum] = df_com[colum].astype('category').cat.codes

***Machine learning***

In [90]:
X = df_com.query('train == 1').drop(['price', 'train', 'price_log2',
                                     ], axis=1)
X_t = df_com.query('train == 0').drop(['price', 'train', 'price_log2',
                                       ], axis=1)
y = df_com.query('train == 1').price

In [91]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=42)

***CatBoost***

***Fit***

In [92]:
cat_boost = CatBoostRegressor(iterations=5000, random_seed=42, eval_metric='MAPE',
                              custom_metric=['R2', 'MAE'], silent=True,)
cat_boost.fit(X_train, y_train, eval_set=(X_test, y_test),
              verbose_eval=0, use_best_model=True, plot=True)

In [93]:
y_pred = cat_boost.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
# Точность модели по метрике MAPE 16.29%

***Log Target***

In [94]:
cat_boost_log = CatBoostRegressor(iterations = 5000,
                          random_seed = 42,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
cat_boost_log.fit(X_train, np.log(y_train),
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         )

In [95]:
predict_test = np.exp(cat_boost_log.predict(X_test))
predict_submission = np.exp(cat_boost_log.predict(X_t))

In [96]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")
# Точность модели по метрике MAPE: 13.36%

***Random Forest***

In [97]:
rf = RandomForestRegressor(random_state=42, n_jobs=-1, verbose=1)
rf.fit(X_train, y_train)
predict_rf = rf.predict(X_test)

print(f"The MAPE mertics of the Random Forest model using MAPE metrics: {(mape(y_test, predict_rf) * 100):0.2f}%.")

rf_log = RandomForestRegressor(random_state=42, n_jobs=-1, verbose=1)
rf_log.fit(X_train, np.log(y_train))
predict_rf_log = np.exp(rf_log.predict(X_test))

print(f"The MAPE mertic for the Random Forest model is : {(mape(y_test, predict_rf_log) * 100):0.2f}%.")

***ExtraTreesRegressor***

In [98]:
etr_log = ExtraTreesRegressor(random_state=42, n_jobs=-1, verbose=1)
etr_log.fit(X_train, np.log(y_train))

In [99]:
y_pred = np.exp(etr_log.predict(X_test))
etr_log_mape_value = mape(y_test, y_pred)
print(f"The MAPE mertic for the default ExtraTreesRegressor model using 4-fold CV is: {(np.mean(etr_log_mape_value) * 100):0.2f}%.")

***XGBoostRegressor***

In [100]:
xgb_log = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.05, max_depth=12, alpha=1,
                           n_estimators=1000, random_state=42, verbose=1, n_jobs=-1,)

In [101]:
xgb_log.fit(X_train, np.log(y_train))

In [102]:
y_pred = np.exp(xgb_log.predict(X_test))
xgb_log_mape_value = mape(y_test, y_pred)
print(f"The MAPE mertic for the XGBRegressor model using 4-fold CV: {(np.mean(xgb_log_mape_value) * 100):0.2f}%.")

***Окончательное решение о модели:***

***Лучшие показатели MAPE по нашим данным в таблице лидеров показала бустерная (XGBoostRegressor) модель.***

***Submission***

In [103]:
predict_submission = np.exp(xgb_log.predict(X_t))
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_final.csv', index=False)
sample_submission.head(10)